In [1]:
import pandas as pd
import plotly.express as px

In [16]:
org_data = pd.read_parquet("escooter_history_2022.parquet")

In [25]:
# holiday (float) -> boolean
# wokringday (float) -> boolean
# registered_customer (boolean) -> float
org_data_clean = org_data.copy()
org_data_clean.holiday = org_data.holiday.astype(bool)
org_data_clean.workingday = org_data.workingday.astype(bool)

In [66]:
df = org_data_clean.copy()

In [84]:
df1 = df.groupby(df['datetime'].dt.date).sum()
df1

,holiday,workingday,temp,atemp,humidity,windspeed,registered_customer
datetime,,,,,,,
2020-01-04,0,0,16242.56,20099.575,78483.0,15982.8768,654
2020-01-05,0,0,11546.42,13487.235,53780.0,12715.9836,670
2020-01-06,0,1349,15038.80,13678.575,54348.0,19863.6508,1229
2020-01-07,0,1562,14138.44,17593.315,88114.0,19293.3602,1454
2020-01-08,0,1600,15438.96,18341.945,61168.0,22581.0904,1518
...,...,...,...,...,...,...,...
2021-12-30,0,4231,44321.82,48722.540,251627.0,94106.6724,4024
2021-12-31,0,8144,84853.60,103898.580,477674.0,84219.2479,7629
2022-01-01,0,0,62593.06,71895.840,452743.0,48865.8135,5293


In [85]:
px.line(df1, x=df1.index, y="registered_customer")